In [5]:
import pandas as pd
import numpy as np
from numpy import loadtxt
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot

In [4]:
all = pd.read_csv('data2/PatientInfo.csv')

In [7]:
all.head()

,sex,age,province,confirmed_date
0,female,30,Incheon,2020-01-20
1,male,50,Seoul,2020-01-23
2,male,50,Gyeonggi-do,2020-01-26
3,male,50,Gyeonggi-do,2020-01-27
4,male,30,Seoul,2020-01-30


In [12]:
weather = pd.read_csv('data2/weather3.csv')

In [13]:
weather.head()

,province,date,avg_temp,avg_relative_humidity
0,Seoul,2020-01-20,0.7,46.8
1,Busan,2020-01-20,5.1,45.6
2,Daegu,2020-01-20,4.6,44.1
3,Gwangju,2020-01-20,4.0,63.1
4,Incheon,2020-01-20,1.2,56.0


In [14]:
time = pd.read_csv('data2/TimeProvince.csv')

In [15]:
time.head()

,date,province,confirmed
0,2020-01-20,Seoul,0
1,2020-01-20,Busan,0
2,2020-01-20,Daegu,0
3,2020-01-20,Incheon,1
4,2020-01-20,Gwangju,0


In [16]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2128 entries, 0 to 2127
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   province               2128 non-null   object 
 1   date                   2128 non-null   object 
 2   avg_temp               2128 non-null   float64
 3   avg_relative_humidity  2128 non-null   float64
dtypes: float64(2), object(2)
memory usage: 66.6+ KB


In [17]:
time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2261 entries, 0 to 2260
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       2261 non-null   object
 1   province   2261 non-null   object
 2   confirmed  2261 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 53.1+ KB


In [18]:
weather['dp'] = weather['date'] + weather['province']

In [22]:
weather=weather.drop('dp', axis=1)

In [23]:
weather

,province,date,avg_temp,avg_relative_humidity
0,Seoul,2020-01-20,0.7,46.8
1,Busan,2020-01-20,5.1,45.6
2,Daegu,2020-01-20,4.6,44.1
3,Gwangju,2020-01-20,4.0,63.1
4,Incheon,2020-01-20,1.2,56.0
...,...,...,...,...
2123,Jeollabuk-do,2020-05-31,20.9,61.8
2124,Jeollanam-do,2020-05-31,19.1,84.1
2125,Gyeongsangbuk-do,2020-05-31,18.5,77.4
2126,Gyeongsangnam-do,2020-05-31,19.3,71.5


In [40]:
df=pd.merge(weather, time, how='inner', on=['date', 'province'])

In [41]:
df

,province,date,avg_temp,avg_relative_humidity,confirmed
0,Seoul,2020-01-20,0.7,46.8,0
1,Busan,2020-01-20,5.1,45.6,0
2,Daegu,2020-01-20,4.6,44.1,0
3,Gwangju,2020-01-20,4.0,63.1,0
4,Incheon,2020-01-20,1.2,56.0,1
...,...,...,...,...,...
2123,Jeollabuk-do,2020-05-31,20.9,61.8,21
2124,Jeollanam-do,2020-05-31,19.1,84.1,19
2125,Gyeongsangbuk-do,2020-05-31,18.5,77.4,1379
2126,Gyeongsangnam-do,2020-05-31,19.3,71.5,123


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2128 entries, 0 to 2127
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   province               2128 non-null   object 
 1   date                   2128 non-null   object 
 2   avg_temp               2128 non-null   float64
 3   avg_relative_humidity  2128 non-null   float64
 4   confirmed              2128 non-null   int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 99.8+ KB


In [29]:
df.isnull().sum()

province                 0
date                     0
avg_temp                 0
avg_relative_humidity    0
confirmed                0
dtype: int64

In [42]:
df

,province,date,avg_temp,avg_relative_humidity,confirmed
0,Seoul,2020-01-20,0.7,46.8,0
1,Busan,2020-01-20,5.1,45.6,0
2,Daegu,2020-01-20,4.6,44.1,0
3,Gwangju,2020-01-20,4.0,63.1,0
4,Incheon,2020-01-20,1.2,56.0,1
...,...,...,...,...,...
2123,Jeollabuk-do,2020-05-31,20.9,61.8,21
2124,Jeollanam-do,2020-05-31,19.1,84.1,19
2125,Gyeongsangbuk-do,2020-05-31,18.5,77.4,1379
2126,Gyeongsangnam-do,2020-05-31,19.3,71.5,123


In [148]:
#x = df.iloc[:, :-1].values
#y = df.iloc[:, 3].values
#z = df.iloc[:, :].values

q = df.drop('date', axis=1)
z = q.iloc[:, :].values
a = pd.DataFrame(z)

In [58]:
le = LabelEncoder()
x[:,0] = le.fit_transform(x[:, 0])

In [59]:
x

array([[14, '2020-01-20', 0.7, 46.8],
       [0, '2020-01-20', 5.1, 45.6],
       [3, '2020-01-20', 4.6, 44.1],
       ...,
       [8, '2020-05-31', 18.5, 77.4],
       [9, '2020-05-31', 19.3, 71.5],
       [11, '2020-05-31', 19.7, 88.1]], dtype=object)

In [63]:
print(x)

[[14 '2020-01-20' 0.7 46.8]
 [0 '2020-01-20' 5.1 45.6]
 [3 '2020-01-20' 4.6 44.1]
 ...
 [8 '2020-05-31' 18.5 77.4]
 [9 '2020-05-31' 19.3 71.5]
 [11 '2020-05-31' 19.7 88.1]]


In [75]:
x

array([['Seoul', '2020-01-20', 0.7, 46.8],
       ['Busan', '2020-01-20', 5.1, 45.6],
       ['Daegu', '2020-01-20', 4.6, 44.1],
       ...,
       ['Gyeongsangbuk-do', '2020-05-31', 18.5, 77.4],
       ['Gyeongsangnam-do', '2020-05-31', 19.3, 71.5],
       ['Jeju-do', '2020-05-31', 19.7, 88.1]], dtype=object)

In [76]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(x)

OneHotEncoder(handle_unknown='ignore')

In [77]:
enc.categories_

[array(['Busan', 'Chungcheongbuk-do', 'Chungcheongnam-do', 'Daegu',
        'Daejeon', 'Gangwon-do', 'Gwangju', 'Gyeonggi-do',
        'Gyeongsangbuk-do', 'Gyeongsangnam-do', 'Incheon', 'Jeju-do',
        'Jeollabuk-do', 'Jeollanam-do', 'Seoul', 'Ulsan'], dtype=object),
 array(['2020-01-20', '2020-01-21', '2020-01-22', '2020-01-23',
        '2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27',
        '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31',
        '2020-02-01', '2020-02-02', '2020-02-03', '2020-02-04',
        '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-08',
        '2020-02-09', '2020-02-10', '2020-02-11', '2020-02-12',
        '2020-02-13', '2020-02-14', '2020-02-15', '2020-02-16',
        '2020-02-17', '2020-02-18', '2020-02-19', '2020-02-20',
        '2020-02-21', '2020-02-22', '2020-02-23', '2020-02-24',
        '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-28',
        '2020-02-29', '2020-03-01', '2020-03-02', '2020-03-03',
        '2020-03-04', '20

In [78]:
enc.transform(x).toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [87]:
z

array([['Seoul', '2020-01-20', 0.7, 46.8, 0],
       ['Busan', '2020-01-20', 5.1, 45.6, 0],
       ['Daegu', '2020-01-20', 4.6, 44.1, 0],
       ...,
       ['Gyeongsangbuk-do', '2020-05-31', 18.5, 77.4, 1379],
       ['Gyeongsangnam-do', '2020-05-31', 19.3, 71.5, 123],
       ['Jeju-do', '2020-05-31', 19.7, 88.1, 15]], dtype=object)

In [88]:
a

,0,1,2,3,4
0,Seoul,2020-01-20,0.7,46.8,0
1,Busan,2020-01-20,5.1,45.6,0
2,Daegu,2020-01-20,4.6,44.1,0
3,Gwangju,2020-01-20,4,63.1,0
4,Incheon,2020-01-20,1.2,56,1
...,...,...,...,...,...
2123,Jeollabuk-do,2020-05-31,20.9,61.8,21
2124,Jeollanam-do,2020-05-31,19.1,84.1,19
2125,Gyeongsangbuk-do,2020-05-31,18.5,77.4,1379
2126,Gyeongsangnam-do,2020-05-31,19.3,71.5,123


In [89]:
# z, a

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
label_x = LabelEncoder()
z[:, 0] = label_x.fit_transform(z[:, 0])


In [90]:
z

array([[14, '2020-01-20', 0.7, 46.8, 0],
       [0, '2020-01-20', 5.1, 45.6, 0],
       [3, '2020-01-20', 4.6, 44.1, 0],
       ...,
       [8, '2020-05-31', 18.5, 77.4, 1379],
       [9, '2020-05-31', 19.3, 71.5, 123],
       [11, '2020-05-31', 19.7, 88.1, 15]], dtype=object)

In [91]:
onehot = OneHotEncoder(categorical_feature=[0])
z = onehot.fit_transform(z).toarray()

TypeError: __init__() got an unexpected keyword argument 'categorical_feature'

In [122]:
transformer = ColumnTransformer([('one_hot_encoder', OneHotEncoder(), ['province'])],remainder='passthrough')

In [127]:
q

,province,avg_temp,avg_relative_humidity,confirmed
0,Seoul,0.7,46.8,0
1,Busan,5.1,45.6,0
2,Daegu,4.6,44.1,0
3,Gwangju,4.0,63.1,0
4,Incheon,1.2,56.0,1
...,...,...,...,...
2123,Jeollabuk-do,20.9,61.8,21
2124,Jeollanam-do,19.1,84.1,19
2125,Gyeongsangbuk-do,18.5,77.4,1379
2126,Gyeongsangnam-do,19.3,71.5,123


In [149]:
z

array([['Seoul', 0.7, 46.8, 0],
       ['Busan', 5.1, 45.6, 0],
       ['Daegu', 4.6, 44.1, 0],
       ...,
       ['Gyeongsangbuk-do', 18.5, 77.4, 1379],
       ['Gyeongsangnam-do', 19.3, 71.5, 123],
       ['Jeju-do', 19.7, 88.1, 15]], dtype=object)

In [150]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
label_x = LabelEncoder()
z[:, 0] = label_x.fit_transform(z[:, 0])

In [145]:
z

array([[14, 0.7, 46.8, 0],
       [0, 5.1, 45.6, 0],
       [3, 4.6, 44.1, 0],
       ...,
       [8, 18.5, 77.4, 1379],
       [9, 19.3, 71.5, 123],
       [11, 19.7, 88.1, 15]], dtype=object)

In [132]:
transformer = ColumnTransformer([('one_hot_encoder', OneHotEncoder(), [0])],remainder='passthrough')

In [140]:
z = transformer.fit_transform(z[:]).toarray()

In [141]:
z

array([[0.000e+00, 0.000e+00, 0.000e+00, ..., 7.000e-01, 4.680e+01,
        0.000e+00],
       [1.000e+00, 0.000e+00, 0.000e+00, ..., 5.100e+00, 4.560e+01,
        0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 4.600e+00, 4.410e+01,
        0.000e+00],
       ...,
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.850e+01, 7.740e+01,
        1.379e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.930e+01, 7.150e+01,
        1.230e+02],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.970e+01, 8.810e+01,
        1.500e+01]])

In [164]:
a = z
a

<2128x19 sparse matrix of type '<class 'numpy.float64'>'
	with 8123 stored elements in Compressed Sparse Row format>

In [153]:
df=df.drop('date', axis=1)

In [154]:
df

,province,avg_temp,avg_relative_humidity,confirmed
0,Seoul,0.7,46.8,0
1,Busan,5.1,45.6,0
2,Daegu,4.6,44.1,0
3,Gwangju,4.0,63.1,0
4,Incheon,1.2,56.0,1
...,...,...,...,...
2123,Jeollabuk-do,20.9,61.8,21
2124,Jeollanam-do,19.1,84.1,19
2125,Gyeongsangbuk-do,18.5,77.4,1379
2126,Gyeongsangnam-do,19.3,71.5,123


In [ ]:
category_onehot=OneHotEncoder(categories=[pro])

In [157]:
df2 = df

In [158]:
df2

,province,avg_temp,avg_relative_humidity,confirmed
0,Seoul,0.7,46.8,0
1,Busan,5.1,45.6,0
2,Daegu,4.6,44.1,0
3,Gwangju,4.0,63.1,0
4,Incheon,1.2,56.0,1
...,...,...,...,...
2123,Jeollabuk-do,20.9,61.8,21
2124,Jeollanam-do,19.1,84.1,19
2125,Gyeongsangbuk-do,18.5,77.4,1379
2126,Gyeongsangnam-do,19.3,71.5,123


In [160]:
laberEncoder_X = LabelEncoder()
z[:,0] = laberEncoder_X.fit_transform(z[:,0])
transform = ColumnTransformer([("province", OneHotEncoder(), [0])], remainder="passthrough")
z = transform.fit_transform(z)
print (z)

  (0, 14)	1.0
  (0, 16)	0.7
  (0, 17)	46.8
  (1, 0)	1.0
  (1, 16)	5.1
  (1, 17)	45.6
  (2, 3)	1.0
  (2, 16)	4.6
  (2, 17)	44.1
  (3, 6)	1.0
  (3, 16)	4.0
  (3, 17)	63.1
  (4, 10)	1.0
  (4, 16)	1.2
  (4, 17)	56.0
  (4, 18)	1.0
  (5, 4)	1.0
  (5, 16)	3.4
  (5, 17)	57.3
  (6, 15)	1.0
  (6, 16)	4.3
  (6, 17)	48.5
  (7, 7)	1.0
  (7, 16)	1.2
  (7, 17)	68.5
  :	:
  (2121, 18)	60.0
  (2122, 2)	1.0
  (2122, 16)	19.9
  (2122, 17)	66.5
  (2122, 18)	146.0
  (2123, 12)	1.0
  (2123, 16)	20.9
  (2123, 17)	61.8
  (2123, 18)	21.0
  (2124, 13)	1.0
  (2124, 16)	19.1
  (2124, 17)	84.1
  (2124, 18)	19.0
  (2125, 8)	1.0
  (2125, 16)	18.5
  (2125, 17)	77.4
  (2125, 18)	1379.0
  (2126, 9)	1.0
  (2126, 16)	19.3
  (2126, 17)	71.5
  (2126, 18)	123.0
  (2127, 11)	1.0
  (2127, 16)	19.7
  (2127, 17)	88.1
  (2127, 18)	15.0


In [161]:
z

<2128x19 sparse matrix of type '<class 'numpy.float64'>'
	with 8123 stored elements in Compressed Sparse Row format>

In [162]:
c = z.toarray()

In [163]:
c

array([[0.000e+00, 0.000e+00, 0.000e+00, ..., 7.000e-01, 4.680e+01,
        0.000e+00],
       [1.000e+00, 0.000e+00, 0.000e+00, ..., 5.100e+00, 4.560e+01,
        0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 4.600e+00, 4.410e+01,
        0.000e+00],
       ...,
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.850e+01, 7.740e+01,
        1.379e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.930e+01, 7.150e+01,
        1.230e+02],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.970e+01, 8.810e+01,
        1.500e+01]])

In [165]:
c[0]

array([ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  1. ,  0. ,  0.7, 46.8,  0. ])

In [166]:
z[0]

<1x19 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [167]:
c

array([[0.000e+00, 0.000e+00, 0.000e+00, ..., 7.000e-01, 4.680e+01,
        0.000e+00],
       [1.000e+00, 0.000e+00, 0.000e+00, ..., 5.100e+00, 4.560e+01,
        0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 4.600e+00, 4.410e+01,
        0.000e+00],
       ...,
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.850e+01, 7.740e+01,
        1.379e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.930e+01, 7.150e+01,
        1.230e+02],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.970e+01, 8.810e+01,
        1.500e+01]])

In [168]:
c[0]

array([ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  1. ,  0. ,  0.7, 46.8,  0. ])

In [169]:
from sklearn.preprocessing import StandardScaler

prepro = make_column_transformer(
    (StandardScaler(), ['avg_temp', 'avg_relative_humidity', 'confirmed'])
)

In [170]:
c

array([[0.000e+00, 0.000e+00, 0.000e+00, ..., 7.000e-01, 4.680e+01,
        0.000e+00],
       [1.000e+00, 0.000e+00, 0.000e+00, ..., 5.100e+00, 4.560e+01,
        0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 4.600e+00, 4.410e+01,
        0.000e+00],
       ...,
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.850e+01, 7.740e+01,
        1.379e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.930e+01, 7.150e+01,
        1.230e+02],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.970e+01, 8.810e+01,
        1.500e+01]])

In [171]:
d = c

In [172]:
d

array([[0.000e+00, 0.000e+00, 0.000e+00, ..., 7.000e-01, 4.680e+01,
        0.000e+00],
       [1.000e+00, 0.000e+00, 0.000e+00, ..., 5.100e+00, 4.560e+01,
        0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 4.600e+00, 4.410e+01,
        0.000e+00],
       ...,
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.850e+01, 7.740e+01,
        1.379e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.930e+01, 7.150e+01,
        1.230e+02],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.970e+01, 8.810e+01,
        1.500e+01]])

In [174]:
c

array([[0.000e+00, 0.000e+00, 0.000e+00, ..., 7.000e-01, 4.680e+01,
        0.000e+00],
       [1.000e+00, 0.000e+00, 0.000e+00, ..., 5.100e+00, 4.560e+01,
        0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 4.600e+00, 4.410e+01,
        0.000e+00],
       ...,
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.850e+01, 7.740e+01,
        1.379e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.930e+01, 7.150e+01,
        1.230e+02],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.970e+01, 8.810e+01,
        1.500e+01]])

In [175]:
c[0]

array([ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  1. ,  0. ,  0.7, 46.8,  0. ])

In [179]:
import numpy as np
from sklearn.preprocessing import normalize

d = np.random.rand(1000)*10
norm1 =d / np.linalg.norm(d)
norm2 = normalize(d[:, np.newaxis], axis=0).ravel()
print(np.all(norm1 == norm2))

True


In [180]:
d

array([3.45073645, 1.32841846, 5.31686932, 3.65648132, 1.07196717,
       8.00942948, 0.76699394, 2.26804885, 2.11304804, 0.66738011,
       9.6602072 , 3.97083079, 9.85842534, 0.06819263, 5.8211754 ,
       7.81956333, 9.96361236, 8.283082  , 6.72472419, 4.61622284,
       3.94466505, 5.20364555, 7.03011568, 9.0271436 , 1.37315932,
       8.11727528, 4.36041636, 0.09292095, 8.94697502, 7.72696049,
       6.76428744, 2.22455838, 0.96308448, 4.58998257, 5.15564571,
       4.14033463, 5.02661837, 9.68805098, 4.94816488, 4.77507567,
       9.05222623, 9.86918526, 0.08327527, 8.47269461, 5.10140422,
       3.99213215, 4.94563214, 6.02924648, 4.29555917, 7.91399895,
       0.39738706, 6.82094407, 1.45001602, 0.98917784, 9.86252182,
       7.82836825, 1.12778814, 9.51359347, 5.26253696, 2.1652787 ,
       4.02275297, 6.02435845, 5.25123645, 2.57859112, 5.08591737,
       0.42997492, 0.48799109, 2.66000247, 2.07054803, 4.30887028,
       9.41991329, 9.78892809, 8.50026551, 3.26061108, 9.12621

In [181]:
len(d)

1000

In [182]:
c

array([[0.000e+00, 0.000e+00, 0.000e+00, ..., 7.000e-01, 4.680e+01,
        0.000e+00],
       [1.000e+00, 0.000e+00, 0.000e+00, ..., 5.100e+00, 4.560e+01,
        0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 4.600e+00, 4.410e+01,
        0.000e+00],
       ...,
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.850e+01, 7.740e+01,
        1.379e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.930e+01, 7.150e+01,
        1.230e+02],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.970e+01, 8.810e+01,
        1.500e+01]])

In [183]:
d=c

In [184]:
d

array([[0.000e+00, 0.000e+00, 0.000e+00, ..., 7.000e-01, 4.680e+01,
        0.000e+00],
       [1.000e+00, 0.000e+00, 0.000e+00, ..., 5.100e+00, 4.560e+01,
        0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 4.600e+00, 4.410e+01,
        0.000e+00],
       ...,
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.850e+01, 7.740e+01,
        1.379e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.930e+01, 7.150e+01,
        1.230e+02],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.970e+01, 8.810e+01,
        1.500e+01]])

In [185]:
d_normed = d/d.max(axis=0)

In [186]:
print(d_normed)

[[0.         0.         0.         ... 0.02661597 0.468      0.        ]
 [1.         0.         0.         ... 0.19391635 0.456      0.        ]
 [0.         0.         0.         ... 0.17490494 0.441      0.        ]
 ...
 [0.         0.         0.         ... 0.70342205 0.774      0.20034869]
 [0.         0.         0.         ... 0.7338403  0.715      0.01787011]
 [0.         0.         0.         ... 0.74904943 0.881      0.00217928]]


In [187]:
d_normed[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 1.        ,
       0.        , 0.02661597, 0.468     , 0.        ])

In [188]:
d_normed[2]

array([0.        , 0.        , 0.        , 1.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.17490494, 0.441     , 0.        ])

In [190]:
d_normed[978]

array([0.        , 0.        , 0.        , 1.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.57034221, 0.541     , 0.92169112])

In [191]:
df

,province,avg_temp,avg_relative_humidity,confirmed
0,Seoul,0.7,46.8,0
1,Busan,5.1,45.6,0
2,Daegu,4.6,44.1,0
3,Gwangju,4.0,63.1,0
4,Incheon,1.2,56.0,1
...,...,...,...,...
2123,Jeollabuk-do,20.9,61.8,21
2124,Jeollanam-do,19.1,84.1,19
2125,Gyeongsangbuk-do,18.5,77.4,1379
2126,Gyeongsangnam-do,19.3,71.5,123


In [192]:
d

array([[0.000e+00, 0.000e+00, 0.000e+00, ..., 7.000e-01, 4.680e+01,
        0.000e+00],
       [1.000e+00, 0.000e+00, 0.000e+00, ..., 5.100e+00, 4.560e+01,
        0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 4.600e+00, 4.410e+01,
        0.000e+00],
       ...,
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.850e+01, 7.740e+01,
        1.379e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.930e+01, 7.150e+01,
        1.230e+02],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.970e+01, 8.810e+01,
        1.500e+01]])

In [193]:
d[0]

array([ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  1. ,  0. ,  0.7, 46.8,  0. ])

In [212]:
d[:, 16:]

array([[7.000e-01, 4.680e+01, 0.000e+00],
       [5.100e+00, 4.560e+01, 0.000e+00],
       [4.600e+00, 4.410e+01, 0.000e+00],
       ...,
       [1.850e+01, 7.740e+01, 1.379e+03],
       [1.930e+01, 7.150e+01, 1.230e+02],
       [1.970e+01, 8.810e+01, 1.500e+01]])

In [244]:
list=['Busan', 'Chungcheongbuk-do', 'Chungcheongnam-do', 'Daegu', 'Daejeon', 'Gangwon-do', 'Gwangju', 'Gyeonggi-do', 'Gyeongsangbuk-do', 'Gyeongsangnam-do', 'Incheon', 'Jeju-do', 'Jeollabuk-do', 'Jeollanam-do','Seoul','Ulsan', 'avg_temp', 'avg_relative_humidity', 'confirmed']
haha=pd.DataFrame(
    data=d[:, :],
    index=[i for i in range(d.shape[0])],
    columns=[i for i in list]
)

In [220]:
d

array([[0.000e+00, 0.000e+00, 0.000e+00, ..., 7.000e-01, 4.680e+01,
        0.000e+00],
       [1.000e+00, 0.000e+00, 0.000e+00, ..., 5.100e+00, 4.560e+01,
        0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 4.600e+00, 4.410e+01,
        0.000e+00],
       ...,
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.850e+01, 7.740e+01,
        1.379e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.930e+01, 7.150e+01,
        1.230e+02],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.970e+01, 8.810e+01,
        1.500e+01]])

In [222]:
wow

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,avg_temp,avg_relative_humidity,confirmed
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.7,46.8,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.1,45.6,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.6,44.1,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,63.1,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.2,56.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,20.9,61.8,21.0
2124,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,19.1,84.1,19.0
2125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.5,77.4,1379.0
2126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,19.3,71.5,123.0


In [240]:
d[15]

array([ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        1. ,  0. ,  0. ,  0. ,  0. ,  8.4, 50.3,  0. ])

In [227]:
df.head(16)

,province,avg_temp,avg_relative_humidity,confirmed
0,Seoul,0.7,46.8,0
1,Busan,5.1,45.6,0
2,Daegu,4.6,44.1,0
3,Gwangju,4.0,63.1,0
4,Incheon,1.2,56.0,1
5,Daejeon,3.4,57.3,0
6,Ulsan,4.3,48.5,0
7,Gyeonggi-do,1.2,68.5,0
8,Gangwon-do,3.7,38.8,0
9,Chungcheongbuk-do,0.9,63.3,0


In [245]:
haha

,Busan,Chungcheongbuk-do,Chungcheongnam-do,Daegu,Daejeon,Gangwon-do,Gwangju,Gyeonggi-do,Gyeongsangbuk-do,Gyeongsangnam-do,Incheon,Jeju-do,Jeollabuk-do,Jeollanam-do,Seoul,Ulsan,avg_temp,avg_relative_humidity,confirmed
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.7,46.8,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.1,45.6,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.6,44.1,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,63.1,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.2,56.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,20.9,61.8,21.0
2124,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,19.1,84.1,19.0
2125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.5,77.4,1379.0
2126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,19.3,71.5,123.0


In [243]:
haha.to_csv('data2/good.csv')

In [249]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

cols=['Busan', 'Chungcheongbuk-do', 'Chungcheongnam-do', 'Daegu', 'Daejeon', 'Gangwon-do', 'Gwangju', 'Gyeonggi-do', 'Gyeongsangbuk-do', 'Gyeongsangnam-do', 'Incheon', 'Jeju-do', 'Jeollabuk-do', 'Jeollanam-do','Seoul','Ulsan', 'avg_temp', 'avg_relative_humidity']
X = haha[cols]
Y = haha.confirmed

x_train, x_valid, y_train, y_valid = train_test_split(X, Y)

my_model = XGBRegressor()
my_model.fit(x_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=16, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [250]:
from sklearn.metrics import mean_absolute_error

prediction = my_model.predict(x_valid)
print("mean absolute error: " + str(mean_absolute_error(prediction, y_valid)))

mean absolute error: 108.60036792080933


In [251]:
my_model = XGBRegressor(n_estimator=500)
my_model.fit(x_train, y_train)

prediction = my_model.predict(x_valid)
print("mean absolute error: " + str(mean_absolute_error(prediction, y_valid)))

[02:55:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


mean absolute error: 108.60036792080933


In [253]:
my_model = XGBRegressor(n_estimator=500)
my_model.fit(x_train, y_train, early_stopping_rounds=5,
            eval_set=[(x_valid, y_valid)],
            verbose=False)

prediction = my_model.predict(x_valid)
print("mean absolute error: " + str(mean_absolute_error(prediction, y_valid)))

[02:58:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


mean absolute error: 118.02724716537877


In [254]:
my_model = XGBRegressor(n_estimator=500, learning_rate=0.05)
my_model.fit(x_train, y_train, early_stopping_rounds=5,
            eval_set=[(x_valid, y_valid)],
            verbose=False)

prediction = my_model.predict(x_valid)
print("mean absolute error: " + str(mean_absolute_error(prediction, y_valid)))

[02:59:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


mean absolute error: 127.87021814163467


In [255]:
my_model = XGBRegressor(n_estimator=500, learning_rate=0.05, n_jobs=4)
my_model.fit(x_train, y_train, early_stopping_rounds=5,
            eval_set=[(x_valid, y_valid)],
            verbose=False)

prediction = my_model.predict(x_valid)
print("mean absolute error: " + str(mean_absolute_error(prediction, y_valid)))

[02:59:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


mean absolute error: 127.87021814163467


In [257]:
my_model = XGBRegressor(n_estimator=100, learning_rate=0.05)
my_model.fit(x_train, y_train, early_stopping_rounds=5,
            eval_set=[(x_valid, y_valid)],
            verbose=False)

prediction = my_model.predict(x_valid)
print("mean absolute error: " + str(mean_absolute_error(prediction, y_valid)))

[03:05:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


mean absolute error: 127.87021814163467


In [258]:
my_model

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimator=100, n_estimators=100, n_jobs=16, num_parallel_tree=1,
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)